In [1]:
import os

def get_validation_yolo(dataset_name):
    data_folder = f"datasets/{dataset_name}_png"
    val_data = os.path.join(data_folder, "val_seg.txt")
    with open(val_data) as fp:
        lines = fp.readlines()
        print([os.path.basename(i)for i in lines])
        patients = set([int(os.path.basename(case).split('_')[0]) for case in lines])
    return sorted(list(patients))

In [2]:
import pandas as pd

def LoadAbus23(dataset_path, label_file):
    dataset = pd.read_csv(os.path.join(dataset_path, label_file))
    print("Dataset columns:", dataset.columns)
    return dataset

In [3]:

from ultralytics import YOLO
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
import cv2

def normalize_8bits(image: np.ndarray):
    return (255.0 *(image - image.min()) / (image.max() - image.min())).astype(np.uint8)

def predict_abus(model, input_abus, mask_abus = None, coef_th = 0.5):
    
    data_array = sitk.GetArrayFromImage(input_abus)
    mask_array = sitk.GetArrayFromImage(mask_abus)
    
    masks = []
     # For each slice
    for idx in range(len(data_array)): #first dimension is z in numpy (z,y,x)
        data_slice = data_array[idx, ...]
        data_size = data_slice.shape
        mask_slice = mask_array[idx, ...]
         
        # Normalize to 8-bits
        data_slice = normalize_8bits(data_slice)
        #data_slice_3 = np.stack([data_slice,data_slice,data_slice]) #np.expand_dims(, axis=0)
        #print( data_slice_3.shape)
        cv2.imwrite("temp.png", data_slice)
        
        results = model("temp.png", verbose=False)[0].cpu().numpy()
        mask_object = results.masks
        
        mask = np.zeros(data_size)
        if mask_object is not None:
            
            mask_data = mask_object.data
            for i in range(mask_data.shape[0]):
                
                pred_conf = results.boxes[i].conf  # confidence score, (N, )
                if pred_conf < coef_th:
                    continue
            
                m = cv2.resize(mask_data[i, ...], dsize=(data_size[1], data_size[0])) # interpolation=cv2.INTER_CUBIC)
                mask = np.logical_or(mask, m).astype("float32")
                
            #plt.figure()
            #f, axarr = plt.subplots(1,2) 
            #axarr[0].set_title(str(conf))
            #axarr[0].imshow(mask_slice)
            #axarr[1].imshow(mask)
                
        if mask.shape != data_size:
            break
        masks.append(mask) #
        
    
    mask_3d = np.stack(masks)
    output_mask = sitk.GetImageFromArray(mask_3d)
    output_mask.CopyInformation(input_abus)
    
    #castImageFilter = sitk.CastImageFilter()
    #castImageFilter.SetOutputPixelType(sitk.sitkFloat32)
    #img = castImageFilter.Execute(img)
    
    return output_mask
    
    
def predict_abus_file(model, input_file, output_file, coef_th = 0.5):

    input_data = sitk.ReadImage(input_file)
    input_mask = sitk.ReadImage(input_file.replace("DATA", "MASK"))
    output_mask = predict_abus(model, input_data, input_mask, coef_th)
    sitk.WriteImage(output_mask, output_file, useCompression=True)
    
    
        


In [4]:
from ultralytics import YOLO
from tqdm import tqdm


yolo_model = '/home/joel/abus23/runs/segment/train10/weights/best.pt'

# ABUS 23
dataset_path = "datasets/Train"
label_file = "labels.csv"
dataset_name = "abus23_25"

output_folder = os.path.join("results_masks", dataset_name, "raw_stack")
os.makedirs(output_folder, exist_ok=True)

# Load dataset info
dataset =  LoadAbus23(dataset_path, label_file)

# Load a model
print("Loading model....")
model = YOLO(yolo_model)  # pretrained YOLOv8n model

# Get validation cases
val = get_validation_yolo(dataset_name)

# Filter validation cases
dataset = dataset[dataset['case_id'].isin(val)]

gt_list = []
pred_list = []
for _, row in tqdm(dataset.iterrows(), total=len(dataset)):

    id = row.case_id
    abus_file = os.path.join(dataset_path, row.data_path.replace('\\','/'))
    output_file = os.path.join(output_folder, f"{id}.nrrd")

    predict_abus_file(model, abus_file, output_file, coef_th=0.6)
    
    gt_list.append(os.path.join(dataset_path, row.mask_path.replace('\\','/')))
    pred_list.append(output_file)


Dataset columns: Index(['case_id', 'label', 'data_path', 'mask_path'], dtype='object')
Loading model....
['022_291_M.png\n', '075_200_M.png\n', '075_195_M.png\n', '075_158_M.png\n', '089_194_M.png\n', '074_177_B.png\n', '022_304_M.png\n', '010_222_M.png\n', '022_297_M.png\n', '009_200_M.png\n', '075_165_M.png\n', '022_311_M.png\n', '022_293_M.png\n', '039_208_B.png\n', '099_280_M.png\n', '022_268_M.png\n', '010_218_M.png\n', '025_181_B.png\n', '039_201_B.png\n', '039_200_B.png\n', '066_236_M.png\n', '074_166_B.png\n', '075_180_M.png\n', '025_176_B.png\n', '067_199_M.png\n', '090_232_M.png\n', '080_219_M.png\n', '066_228_M.png\n', '025_170_B.png\n', '007_264_B.png\n', '031_262_B.png\n', '080_216_M.png\n', '074_171_B.png\n', '074_158_B.png\n', '075_189_M.png\n', '066_240_M.png\n', '080_237_M.png\n', '025_193_B.png\n', '014_106_B.png\n', '090_242_M.png\n', '022_308_M.png\n', '066_217_M.png\n', '067_212_M.png\n', '075_156_M.png\n', '099_274_M.png\n', '075_175_M.png\n', '099_277_M.png\n', '

100%|██████████| 20/20 [05:47<00:00, 17.39s/it]


In [5]:
import SimpleITK

def score_case(gt_path, pred_path):
    # Load the images for this case
    gt = SimpleITK.ReadImage(gt_path)
    pred = SimpleITK.ReadImage(pred_path)

    # Cast to the same type
    caster = SimpleITK.CastImageFilter()
    caster.SetOutputPixelType(SimpleITK.sitkUInt8)
    caster.SetNumberOfThreads(1)
    gt = caster.Execute(gt)
    pred = caster.Execute(pred)

    # Score the case
    overlap_measures = SimpleITK.LabelOverlapMeasuresImageFilter()
    overlap_measures.SetNumberOfThreads(1)
    overlap_measures.Execute(gt, pred)
    hausdorff_filter = SimpleITK.HausdorffDistanceImageFilter()
    hausdorff_filter.Execute(gt, pred)

    return {
            'DiceCoefficient': overlap_measures.GetDiceCoefficient(),
            'HDCoefficient': hausdorff_filter.GetHausdorffDistance(),
            'score': overlap_measures.GetDiceCoefficient()-hausdorff_filter.GetHausdorffDistance(),
        }

dice = []
for p,g in zip(pred_list, gt_list):
    score = score_case(g, p)
    print("Case:", p, "     Dice:", score)
    dice.append(score['DiceCoefficient'])
    
print("dice", np.array(dice).mean())
    

Case: results_masks/abus23_25/raw_stack/7.nrrd      Dice: {'DiceCoefficient': 0.4108192408071463, 'HDCoefficient': 293.4177908716511, 'score': -293.00697163084396}
Case: results_masks/abus23_25/raw_stack/9.nrrd      Dice: {'DiceCoefficient': 0.0, 'HDCoefficient': 230.59488285736091, 'score': -230.59488285736091}
Case: results_masks/abus23_25/raw_stack/10.nrrd      Dice: {'DiceCoefficient': 0.5383357066362257, 'HDCoefficient': 226.57890457851542, 'score': -226.0405688718792}
Case: results_masks/abus23_25/raw_stack/14.nrrd      Dice: {'DiceCoefficient': 0.13659776114277866, 'HDCoefficient': 340.74917461382057, 'score': -340.6125768526778}
Case: results_masks/abus23_25/raw_stack/22.nrrd      Dice: {'DiceCoefficient': 0.5968216750111011, 'HDCoefficient': 57.706152185014034, 'score': -57.109330510002934}
Case: results_masks/abus23_25/raw_stack/25.nrrd      Dice: {'DiceCoefficient': 0.6108740813545691, 'HDCoefficient': 45.221676218380054, 'score': -44.61080213702549}


RuntimeError: Exception thrown in SimpleITK HausdorffDistanceImageFilter_Execute: /tmp/SimpleITK-build/ITK-prefix/include/ITK-5.3/itkDirectedHausdorffDistanceImageFilter.hxx:144:
ITK ERROR: pixelcount is equal to 0